In [2]:
import findspark
findspark.init()

import datetime as dt
import pymysql
import pandas.io.sql as psql
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, month,year,dayofmonth
import regex as re
import pyinputplus as pyip


In [ ]:
spark = SparkSession.builder.master("local[1]").appName('Credit_card.com').getOrCreate()

In [3]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='creditcard_capstone'
)


In [4]:
cursor = connection.cursor()

In [ ]:
 Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.

In [ ]:
sql = "SELECT c.CUST_ZIP,c.SSN,cc.CUST_SSN,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.TIMEID FROM cdw_sapp_customer c  JOIN cdw_sapp_credit_card cc ON c.SSN=cc.CUST_SSN "
cursor.execute(sql)
result_set = cursor.fetchall()
data_frame = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

In [ ]:
def transaction_mod1():
    sql = "SELECT c.CUST_ZIP,c.SSN,cc.CUST_SSN,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.TIMEID FROM cdw_sapp_customer c  JOIN cdw_sapp_credit_card cc ON c.SSN=cc.CUST_SSN "
    cursor.execute(sql)
    result_set = cursor.fetchall()
    data_frame = psql.read_sql(sql, con=connection)
    pd.set_option("display.max_columns", None)      
    zipcode = 21042 
    month = 11
    year = 2018
    customer_transaction = data_frame[(data_frame.CUST_ZIP == zipcode) &
                                      (data_frame.TIMEID.dt.month == month) &
                                      (data_frame.TIMEID.dt.year == year)]
    daily_values = customer_transaction.groupby(customer_transaction.TIMEID.dt.day)["TRANSACTION_VALUE"].sum()
    result =pd.DataFrame({"Day": daily_values.index, "Transaction Value": daily_values.values})
    result = result.sort_values("Day", ascending=False)
    print(result)


 Used to display the number and total values of transactions for a given type.

In [ ]:
def transaction_mod2():
    transaction_type = 'Bills'
    transaction_type = data_frame[(data_frame['TRANSACTION_TYPE']=='Bills')&(data_frame['TRANSACTION_VALUE'])]
    transaction_df = transaction_type.groupby('TRANSACTION_TYPE').agg(trans_count=('TRANSACTION_TYPE', 'count'), 
                                                  trans_val=('TRANSACTION_VALUE', 'sum'))
    transaction_df['trans_val'] = round(transaction_df['trans_val'], 2)
    print(transaction_df)

transaction_mod2()

Used to display the total number and total values of transactions for branches in a given state.

In [ ]:
def transaction_mod3():
        sql = "SELECT bc.BRANCH_CODE,bc.BRANCH_STATE,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.BRANCH_CODE FROM cdw_sapp_branch bc JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE=cc.BRANCH_CODE"
        cursor.execute(sql)
        result_set = cursor.fetchall()
        data_frame = psql.read_sql(sql, con=connection)
        pd.set_option("display.max_columns", None)
        branch_state = 'PA'
        branch_transaction = data_frame[(data_frame['BRANCH_STATE']=='PA')&(data_frame['TRANSACTION_VALUE'])]
        trans_by_branch_state = branch_transaction.groupby([ 'BRANCH_STATE']).agg(total_transactions=('TRANSACTION_VALUE', 'sum'))
        print(trans_by_branch_state)

transaction_mod3()

Used to check the existing account details of a customer.

In [ ]:
sql = "SELECT * FROM cdw_sapp_customer "
cursor.execute(sql)
result_set = cursor.fetchall()
data_frame = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

In [ ]:
data_frame["TIMEID"] = pd.to_datetime(data_frame["TIMEID"],format='%Y%m%d')

In [ ]:
def customer_account():
    selected_cols = ['SSN', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'CUST_COUNTRY', 'CUST_CITY', 'CUST_EMAIL',
                 'CUST_PHONE', 'FULL_STREET_ADDRESS', 'CUST_STATE', 'CUST_ZIP', 'CREDIT_CARD_NO']
    ssn = 123456100
    result_df = data_frame.loc[data_frame['SSN'] == ssn, selected_cols]
    print(result_df)

Used to modify the existing account details of a customer.

In [ ]:
def module_two():
    spark_df = spark.createDataFrame(data_frame)
    # todo get user input
    print('Module Two:\n\tModify existing account details of a customer by entering customers credit card number\n')
    ccn = 4210653310102868
    mod2 = spark_df.select('SSN', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'CUST_COUNTRY', 'CUST_CITY', 'CUST_EMAIL',
                     'CUST_PHONE', 'FULL_STREET_ADDRESS', 'CUST_STATE', 'CUST_ZIP', 'CREDIT_CARD_NO').filter(
        spark_df.CREDIT_CARD_NO == ccn)
    mod2.distinct().show()
    # todo ask user what to modify
    column = input('Enter Column name value is stored in: ')
    replace = input('Enter current value from the column {}: '.format(column))  # replace with current value
    new_entry = input('Enter new value to replace old value: ')
    mod2.withColumn(column, re.replace(column, replace, new_entry)).distinct() \
        .show()

Used to generate a monthly bill for a credit card number for a given month and year.

In [ ]:
sql = "SELECT * FROM cdw_sapp_credit_card "
cursor.execute(sql)
result_set = cursor.fetchall()
data_frame = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

In [ ]:
data_frame["TIMEID"] = pd.to_datetime(data_frame["TIMEID"],format='%Y%m%d')

In [ ]:
spark_df= spark_df.withColumn("TIMEID", to_date("TIMEID", "yyyy-MM-dd"))
spark_df= spark_df.withColumn("month", month("TIMEID"))
spark_df= spark_df.withColumn("year", year("TIMEID"))
spark_df= spark_df.withColumn("days", dayofmonth("TIMEID"))

In [ ]:
def module_three():
    spark_df = spark.createDataFrame(data_frame)
    spark_df= spark_df.withColumn("TIMEID", to_date("TIMEID", "yyyy-MM-dd"))
    spark_df= spark_df.withColumn("month", month("TIMEID"))
    spark_df= spark_df.withColumn("year", year("TIMEID"))
    spark_df= spark_df.withColumn("days", dayofmonth("TIMEID"))
    # todo get user input
    print('Module Three:\n\tGenerate a monthly bill for a credit card number for a given month and year')
    ccn = 123456698
    month = 12
    year = 2018
    mod3 = spark_df.select('CUST_SSN', 'TIMEID','TRANSACTION_TYPE',
                     'TRANSACTION_VALUE').filter(
                        spark_df.CUST_SSN == ccn).\
                        filter((spark_df.month == month)& (spark_df.year== year))
    mod3.show()
    print('Credit Card Statement for Month:{} Year:{} '.format(month, year))
    mod3.agg({'TRANSACTION_VALUE': 'sum'}).show()

Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order

In [ ]:
def module_four():
     spark_df = spark.createDataFrame(data_frame)
     spark_df= spark_df.withColumn("TIMEID", to_date("TIMEID", "yyyy-MM-dd"))
     spark_df= spark_df.withColumn("month", month("TIMEID"))
     spark_df= spark_df.withColumn("year", year("TIMEID"))
     spark_df= spark_df.withColumn("days", dayofmonth("TIMEID"))

     start_date = 2018-3-11
     end_date = 2018-12-20
     customer_ssn = 123456698

     mod4 = spark_df.select('TIMEID', 'TRANSACTION_TYPE', 'TRANSACTION_VALUE','CUST_SSN') \
         .filter(spark_df.CUST_SSN == customer_ssn) \
         .filter((spark_df.TIMEID==start_date)&(spark_df.TIMEID==end_date)) \
         .orderBy(year(spark_df.TIMEID).desc(), month(spark_df.TIMEID).desc(), dayofmonth(spark_df.TIMEID).desc())
     mod4.show()

In [ ]:
print("\n CUstomer and Transaction detail modules")

In [5]:
while True:
    print("\nMENU")
    print("1.Transaction Details OF A Customer")
    print("2. Customer deatails with the given option")
    print("3.Quit")
    choice = pyip.inputInt("Enter a choice Transaction details or Customer details",min=1,max=3)
    if choice == 1:
        while True:
            print("\nSelect an option from the sub_menu")
            print("1. The transactions made by customers living in a given zip code for a given month and year. Order by day in descending order")
            print("2. The number and total values of transactions for a given type")
            print("3. The total number and total values of transactions for branches in a given state.")
            print("4. Quit")
            sub_choice = pyip.inputInt("Select an option to check the transaction details",min=1,max=4)
            if sub_choice == 1:
                transaction_mod1()
            elif sub_choice == 2:
                transaction_mod2()
            elif sub_choice == 3:
                transaction_mod3()
            else:
                break
    elif choice == 2:
        while True:
            print("\nSelect an option from the sub_menu")
            print("1. Used to check the existing account details of a customer.")
            print("2.  Used to modify the existing account details of a customer.")
            print("3. Used to generate a monthly bill for a credit card number for a given month and year.")
            print("4. Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order")
            sub_choice = pyip.inputInt("Select an option to Get the Customer details",min=1,max=4)
            if sub_choice == 1:
                customer_account()
            elif sub_choice == 2:
                module_two()
            elif sub_choice == 3:
                module_three()
            elif sub_choice == 4:
                module_four()
            else:
                break

    else:
        break
                



MENU
1.Transaction Details OF A Customer
2. Customer deatails with the given option
3.Quit
Enter a choice Transaction details or Customer details